In [1]:
import os
from smolagents import AzureOpenAIServerModel, TransformersModel

MODE = 'azure'  # or 'azure'
if MODE == 'azure':
    model = AzureOpenAIServerModel(
        model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
        api_version=os.environ.get("OPENAI_API_VERSION", "")
    )
if MODE == 'local':
    model = TransformersModel(
        model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    )

In [2]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "pre_medications").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [ ]:
from medminer.task.medication import medication_task

for doc in docs:
    medication_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: medication                                                                                           │
│ Prompt:                                                                                                         │
│     Given a list of medications, save all medications for the patient as csv. The medications can be in any     │
│ language. The medications are usually in the format of a list of medications, with dosage and unit as well as   │
│ their dosage timings (e.g. 1-0-1-0 for moring and evening but not noon or night). Every medication should have  │
│ a single row, if there are multiple medications that can be extracted from a single piece of text, e.g. a       │
│ combined medication, split them up. These are the steps you should follow to complete the task:                 │
│                                                                                                                 │
│     To complete the task make the following steps:                                                              │
│     1. Extract the medications from the text. The medication can be in any language. This is the                │
│ medication_reference column.                                                                                    │
│     2. If the medication name is not in english, translate it to english and infer the medication_translated    │
│ column. Correct any misspellings in the process. Use the following format "Brand name or medication name        │
│ (active ingredient)". e.g. "Aspirin (acetylsalicylic acid)".                                                    │
│     3. Extract the active ingredient of the medication and loose everything that is not relevant. This is the   │
│ active_ingredient column.                                                                                       │
│     4. Extract the dose of the medication. This should only contain the numeric value. This is the dose column. │
│     5. Extract the unit of the dose (e.g. ml, mg, ...). If possible, use mg. If not applicable, write an empty  │
│ string. This is the unit column.                                                                                │
│     6. Extract the dosage timings of the medication. This is the dosage_morning, dosage_noon, dosage_evening    │
│ and dosage_night columns. If you see that the patient is not taking a medication at a certain time point, write │
│ a 0. If you are unsure or missing the information, leave empty.                                                 │
│     7. Extract the dosage information of the medication. This is the dosage_information column. You can write   │
│ any further information here that does not fit the description before. If you see an indication that this might │
│ be a medication that is taken "as needed" write "as needed" here. If you weren't able to infer a medication     │
│ schedule in the task before, write "schedule unkown". If not applicable, write an empty string.                 │
│     8. get the RXCUI for all medications. Use the active ingredient of the medications. If there are multiple   │
│ RXCUI codes, choose the one that fits the best to the translated medication. Usually, the first candidate with  │
│ a score of 1 is the best choice, but you can decide otherwise if you have reasonable grounds for another        │
│ decision. If there are no codes, write an empty string.                                                         │
│     9. get the ATC code for all medications. Use the rxcui of the medications.                                  │
│     10. save the medication information as csv with the columns defined below.                                  │
│                                                       

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'extract_medication_data' with arguments: {'data': [{'patient_id': 5, 'medication_name':          │
│ 'Amlodipin', 'medication_name_corrected': 'Amlodipine (amlodipine)', 'dose': 10, 'unit': 'mg',                  │
│ 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''},       │
│ {'patient_id': 5, 'medication_name': 'Bisoprolol', 'medication_name_corrected': 'Bisoprolol (bisoprolol)',      │
│ 'dose': 10, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0,        │
│ 'dosage_information': ''}, {'patient_id': 5, 'medication_name': 'Lisinopril', 'medication_name_corrected':      │
│ 'Lisinopril (lisinopril)', 'dose': 30, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening':   │
│ 0, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 5, 'medication_name': 'Torasemid',              │
│ 'medication_name_corrected': 'Torasemide (torasemide)', 'dose': 10, 'unit': 'mg', 'dosage_morning': 1,          │
│ 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 5,          │
│ 'medication_name': 'Midazolam', 'medication_name_corrected': 'Midazolam (midazolam)', 'dose': 0, 'unit': '',    │
│ 'dosage_morning': 0, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': 'as        │
│ needed'}, {'patient_id': 5, 'medication_name': 'Simvastatin', 'medication_name_corrected': 'Simvastatin         │
│ (simvastatin)', 'dose': 20, 'unit': 'mg', 'dosage_morning': 0, 'dosage_noon': 0, 'dosage_evening': 1,           │
│ 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 5, 'medication_name': 'Ampicillin',                │
│ 'medication_name_corrected': 'Ampicillin (ampicillin)', 'dose': 4500, 'unit': 'mg', 'dosage_morning': 1,        │
│ 'dosage_noon': 1, 'dosage_evening': 1, 'dosage_night': 1, 'dosage_information': ''}, {'patient_id': 5,          │
│ 'medication_name': 'Sulbactam', 'medication_name_corrected': 'Sulbactam (sulbactam)', 'dose': 4500, 'unit':     │
│ 'mg', 'dosage_morning': 1, 'dosage_noon': 1, 'dosage_evening': 1, 'dosage_night': 1, 'dosage_information':      │
│ ''}]}                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'patient_id': 5, 'medication_name': 'Amlodipin', 'medication_name_corrected': 'Amlodipine 
(amlodipine)', 'dose': 10, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 
'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 5, 'medication_name': 'Bisoprolol', 
'medication_name_corrected': 'Bisoprolol (bisoprolol)', 'dose': 10, 'unit': 'mg', 'dosage_morning': 1, 
'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 5, 
'medication_name': 'Lisinopril', 'medication_name_corrected': 'Lisinopril (lisinopril)', 'dose': 30, 'unit': 'mg', 
'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 'dosage_information': ''}, 
{'patient_id': 5, 'medication_name': 'Torasemid', 'medication_name_corrected': 'Torasemide (torasemide)', 'dose': 
10, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 0, 'dosage_evening': 0, 'dosage_night': 0, 
'dosage_information': ''}, {'patient_id': 5, 'medication_name': 'Midazolam', 'medication_name_corrected': 
'Midazolam (midazolam)', 'dose': 0, 'unit': '', 'dosage_morning': 0, 'dosage_noon': 0, 'dosage_evening': 0, 
'dosage_night': 0, 'dosage_information': 'as needed'}, {'patient_id': 5, 'medication_name': 'Simvastatin', 
'medication_name_corrected': 'Simvastatin (simvastatin)', 'dose': 20, 'unit': 'mg', 'dosage_morning': 0, 
'dosage_noon': 0, 'dosage_evening': 1, 'dosage_night': 0, 'dosage_information': ''}, {'patient_id': 5, 
'medication_name': 'Ampicillin', 'medication_name_corrected': 'Ampicillin (ampicillin)', 'dose': 4500, 'unit': 
'mg', 'dosage_morning': 1, 'dosage_noon': 1, 'dosage_evening': 1, 'dosage_night': 1, 'dosage_information': ''}, 
{'patient_id': 5, 'medication_name': 'Sulbactam', 'medication_name_corrected': 'Sulbactam (sulbactam)', 'dose': 
4500, 'unit': 'mg', 'dosage_morning': 1, 'dosage_noon': 1, 'dosage_evening': 1, 'dosage_night': 1, 
'dosage_information': ''}]

[Step 1: Duration 7.32 seconds| Input tokens: 3,713 | Output tokens: 592]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_atc' with arguments: {'rxcuis': ['381001']}                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: {'381001': {}}

[Step 7: Duration 1.52 seconds| Input tokens: 37,800 | Output tokens: 1,122]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━